# Setup

In [3]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.offline as py
import plotly.graph_objs as go
import pytest as pt

from datetime import datetime

In [4]:
# set the file name. This needs to be done manually, as Jupyter makes it oddly difficult to query the current notebook filename.
__file__ = 'comparer.ipynb'

import ipytest.magics
import pytest

In [5]:
np.set_printoptions(suppress=True)

In [6]:
%%HTML
<style>
    .container { width:100% !important; } 
</style>

In [7]:
# matplotlib configuration
%matplotlib inline
matplotlib.style.use('bmh')

# Common Plotly configuration
py.init_notebook_mode(connected=True)
config={
    'showLink': False,
    'modeBarButtonsToRemove': ['sendDataToCloud', 'hoverClosestCartesian', 'hoverCompareCartesian'],
}

In [8]:
def line_chart(y, x=None):
    py.iplot([{'y':y, 'x':x}], config=config)

# Assumptions

In [9]:
cpi = 0.02
km_per_year = 15000
cost_per_tyre = 350
tyre_life_km = 45000
fuel_cost_per_litre = 1.50
fuel_efficiency_litres_per_100km = 7.0

purchase_price = 35000
age_at_purchase = 0
years_to_model = 15

# Depreciation

## Tests

In [12]:
%%run_pytest[clean] -v --tb=line

def decelerating(y):
    if y < 3:
        return 0.15
    elif y < 5:
        return 0.1
    else:
        return 0.08
    
flat_rate_15 = FlatRate(0.15)
    
def test_depreciated_value_result_sizes():
    iv = 100  # initial value
    y = 5  # num years
    ia = 0  # initial age
    dv, loss = calc_depreciation(iv, years=y, initial_age=ia, dep_rate_func=flat_rate_15)
    assert len(dv) == y
    assert len(loss) == y
    
def test_depreciated_value_first_year():
    iv = 100
    y = 5
    ia = 0  # initial age
    dv, loss = calc_depreciation(iv, years=y, initial_age=ia, dep_rate_func=flat_rate_15)
    assert dv[0] == pt.approx(iv)
    
def test_depreciated_value_second_year():
    iv = 1000
    y = 5
    ia = 0  # initial age
    dv, _ = calc_depreciation(iv, years=y, initial_age=ia, dep_rate_func=flat_rate_15)
    assert dv[1] == pt.approx(iv * (1 - flat_rate_15(0)))
    
def test_depreciated_value():
    iv = 100
    y = 5
    ia = 0
    dv, _ = calc_depreciation(iv, years=y, initial_age=ia, dep_rate_func=flat_rate_15)
    r = 1 - flat_rate_15(0)
    assert dv[0] == pt.approx(iv)
    assert dv[1] == pt.approx(dv[0] * r)
    assert dv[2] == pt.approx(dv[1] * r)
    assert dv[3] == pt.approx(dv[2] * r)
    assert dv[4] == pt.approx(dv[3] * r)
    
def test_loss_vs_dep_value():
    iv = 100
    y = 5
    ia = 0
    dv, loss = calc_depreciation(iv, years=y, initial_age=ia, dep_rate_func=decelerating)
    for i in range(y):
        assert loss[i] == pt.approx(decelerating(i) * dv[i])
    
def test_cumulative_loss():
    iv = 15000
    y = 10
    ia = 0
    dv, loss = calc_depreciation(iv, years=y, initial_age=ia, dep_rate_func=decelerating)
    assert dv[0] - dv[-1] == pt.approx(loss[0:-1].sum())
    

============================================================ test session starts =============================================================
platform linux -- Python 3.6.5, pytest-3.7.4, py-1.6.0, pluggy-0.7.1 -- /home/daniel/.virtualenvs/ccc/bin/python
cachedir: .pytest_cache
rootdir: /home/daniel/code/car-cost-calculator, inifile:
plugins: sugar-0.9.1
collecting ... collected 6 items

comparer.py::test_depreciated_value_result_sizes <- <ipython-input-12-697ad6a36427> PASSED                                             [ 16%]
comparer.py::test_depreciated_value_first_year <- <ipython-input-12-697ad6a36427> PASSED                                               [ 33%]
comparer.py::test_depreciated_value_second_year <- <ipython-input-12-697ad6a36427> PASSED                                              [ 50%]
comparer.py::test_depreciated_value <- <ipython-input-12-697ad6a36427> PASSED                                                          [ 66%]
comparer.py::test_loss_vs_dep_value <- <

# Compound Interest
Solves $A = P(1+\frac{r}{n})^{nt}$ for
* P = principal
* r = annual rate
* n = number of times interest is compounded per year
* t = number of years to compound

Returns an array giving the new principal for each year in the sequence.

In [13]:
def compound_interest(
    principal,
    annual_rate,
    years,
    compounds_per_year=1,
):
    return np.array([principal * ((1 + annual_rate/compounds_per_year)**(yr*compounds_per_year)) for yr in range(years)])

In [14]:
%%run_pytest[clean] -v --tb=line

def test_compound_interest_num_results():
    p = compound_interest(8000, 0.073, 10, 1)
    assert 10 == len(p)
    
def test_compound_interest():
    actual = compound_interest(8000, 0.073, 5, 1)
    expected = [8000.0, 8584.0, 9210.632, 9883.008, 10604.468]
    np.testing.assert_allclose(actual, expected, verbose=True)
    

============================================================ test session starts =============================================================
platform linux -- Python 3.6.5, pytest-3.7.4, py-1.6.0, pluggy-0.7.1 -- /home/daniel/.virtualenvs/ccc/bin/python
cachedir: .pytest_cache
rootdir: /home/daniel/code/car-cost-calculator, inifile:
plugins: sugar-0.9.1
collecting ... collected 2 items

comparer.py::test_compound_interest_num_results <- <ipython-input-14-0d04cf4e4a7c> PASSED                                              [ 50%]
comparer.py::test_compound_interest <- <ipython-input-14-0d04cf4e4a7c> PASSED                                                          [100%]

============================================================== warnings summary ==============================================================
<undetermined location>
  Module already imported so cannot be rewritten: pytest_sugar

-- Docs: https://docs.pytest.org/en/latest/warnings.html
===================================

# Fuel Costs

In [15]:
def fuel_used(distance_km, litres_per_100km):
    return distance_km * litres_per_100km / 100.0

In [16]:
def yearly_fuel_cost(km_per_year, fuel_efficiency, years, cpi, initial_fuel_cost):
    assert cpi >= 0
    assert cpi <= 1.0
    
    fuel_used_per_year = fuel_used(km_per_year, fuel_efficiency)
    indexed_fuel_cost_per_litre = compound_interest(principal=initial_fuel_cost, annual_rate=cpi, years=years, compounds_per_year=1)
    return indexed_fuel_cost_per_litre * fuel_used_per_year

In [17]:
%%run_pytest[clean] -v --tb=line

def test_fuel_used_100km():
    assert fuel_used(100, 8) == 8.0
    
def test_fuel_used_15000km():
    assert fuel_used(15000, 15.8) == 2370.0
    
def test_fuel_cost_over_10years():
    expected = [1890.0, 1929.69, 1970.214, 2011.588, 2053.831, 2096.962, 2140.998, 2185.959, 2231.864, 2278.733]
    actual = yearly_fuel_cost(km_per_year=15000, fuel_efficiency=8.4, years=10, cpi=2.1/100.0, initial_fuel_cost=1.5)
    np.testing.assert_allclose(actual, expected, atol=0.001)

============================================================ test session starts =============================================================
platform linux -- Python 3.6.5, pytest-3.7.4, py-1.6.0, pluggy-0.7.1 -- /home/daniel/.virtualenvs/ccc/bin/python
cachedir: .pytest_cache
rootdir: /home/daniel/code/car-cost-calculator, inifile:
plugins: sugar-0.9.1
collecting ... collected 3 items

comparer.py::test_fuel_used_100km <- <ipython-input-17-bd08d6d8f5fc> PASSED                                                            [ 33%]
comparer.py::test_fuel_used_15000km <- <ipython-input-17-bd08d6d8f5fc> PASSED                                                          [ 66%]
comparer.py::test_fuel_cost_over_10years <- <ipython-input-17-bd08d6d8f5fc> PASSED                                                     [100%]

============================================================== warnings summary ==============================================================
<undetermined location>
  Module alrea

# Running Costs
Costs that depend on distance travelled.

* service_cost = 400,
* service_interval_km = 10000,
* service_interval_years = 1.0,

## To Do: move fuel cost into `RunningCosts` class

In [18]:
class RunningCosts():
    pass

# Standing Costs
Costs that do not depend on distance travelled.

In [19]:
class StandingCosts():
    '''Represents the static costs, which are not dependent on distance travelled'''
    
    def __init__(
        self,
        years = 1,
        cpi = 0.02,
        insurance_per_year = 500,
        registration_per_year = 500,
        roadside_assist_per_year = 200):
        '''Initialise the standing costs object'''
        self.insurance_per_year = compound_interest(principal=insurance_per_year, annual_rate=cpi, years=years, compounds_per_year=1)
        self.registration_per_year = compound_interest(principal=registration_per_year, annual_rate=cpi, years=years, compounds_per_year=1)
        self.roadside_assist_per_year = compound_interest(principal=roadside_assist_per_year, annual_rate=cpi, years=years, compounds_per_year=1)

In [20]:
%%run_pytest[clean] -v --tb=line

def test_standing_costs_1_year():
    actual = StandingCosts(years=1, cpi=0.02, insurance_per_year=100, registration_per_year=200, roadside_assist_per_year=150)
    np.testing.assert_allclose(100, actual.insurance_per_year)
    np.testing.assert_allclose(200, actual.registration_per_year)
    np.testing.assert_allclose(150, actual.roadside_assist_per_year)
    
def test_standing_costs_insurance_3_years():
    actual = StandingCosts(years=3, cpi=0.02, insurance_per_year=100, registration_per_year=200, roadside_assist_per_year=150)
    expected = compound_interest(principal=100, annual_rate=0.02, years=3)
    np.testing.assert_allclose(expected, actual.insurance_per_year)
    
def test_standing_costs_rego_10_years():
    actual = StandingCosts(years=10, cpi=0.02, insurance_per_year=100, registration_per_year=500, roadside_assist_per_year=150)
    expected = compound_interest(principal=500, annual_rate=0.02, years=10)
    np.testing.assert_allclose(expected, actual.registration_per_year)
    

============================================================ test session starts =============================================================
platform linux -- Python 3.6.5, pytest-3.7.4, py-1.6.0, pluggy-0.7.1 -- /home/daniel/.virtualenvs/ccc/bin/python
cachedir: .pytest_cache
rootdir: /home/daniel/code/car-cost-calculator, inifile:
plugins: sugar-0.9.1
collecting ... collected 3 items

comparer.py::test_standing_costs_1_year <- <ipython-input-20-1d60254654f3> PASSED                                                      [ 33%]
comparer.py::test_standing_costs_insurance_3_years <- <ipython-input-20-1d60254654f3> PASSED                                           [ 66%]
comparer.py::test_standing_costs_rego_10_years <- <ipython-input-20-1d60254654f3> PASSED                                               [100%]

============================================================== warnings summary ==============================================================
<undetermined location>
  Module alrea